# Building a simple RAG
We have learned that ChatGPT knows very little about ITU. so let's incorporate some knowledge as a RAG!

For the sake of simplicity, we use LlamaIndex. But you could also build something manually (as most companies do) based on a custom RAG architecture (i.e., programming the communication between LLM and Vector DB/SQL DB yourself).

Either way, we are aiming for a basic setup like this:
![Typical RAG pipeline](RAG_pipeline.png)

## Load external data
Using pre-built class for directory reading.

In [1]:
from llama_index.core import SimpleDirectoryReader

In [8]:
documents = SimpleDirectoryReader("preprocessed").load_data()

### Build Vector DB

In [15]:
import os

# first, load and set openaikey from a txt file I stored it in
with open('oaikey.txt') as keyfile:
    oaikey = keyfile.read().strip()
    
os.environ["OPENAI_API_KEY"] = oaikey

### Set up Vector database (ChromaDB)

In [12]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings

import chromadb

#### Initite chroma db and collection of vectors


In [ ]:
# chroma_client = chromadb.EphemeralClient() # this would only be in RAM
chroma_client = chromadb.PersistentClient(path="./lecture_db/chroma") # client stored on disk
chroma_collection = chroma_client.get_or_create_collection(name="itu_db")

#### Wrap in LLamaIndex objects for easier handling/compatability

In [25]:
# ... basically, we are just specifying the storage to be used as the ChromaDB
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#### Specify the embedding model 

In [26]:
from llama_index.embeddings.openai import OpenAIEmbedding

# let's use OpenAI out-of-the box embeddings.
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

### Start building the database from our config!
Note that you could add multiple processing steps here, such as:
- Using an [ingestion pipeline](https://docs.llamaindex.ai/en/stable/module_guides/loading/ingestion_pipeline/) for further preprocessing
- [modifying chunk size and overlap](https://docs.llamaindex.ai/en/stable/optimizing/basic_strategies/basic_strategies/#chunk-sizes) or introduce specific chunking strategy
- and others

In [ ]:
# You could modify chunk size and overlap like this
# Settings.chunk_size = 512
# Settings.chunk_overlap = 50

In [41]:
from llama_index.core import VectorStoreIndex
# now let's build an index for the database using pre-built functionality
# - Chunk the documents
# - Retrieve embeddings for document chunks
# - Create nodes in db based on docs/chunks
# - Index database for fast retrieval
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model,
    show_progress=True
)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/65 [00:00<?, ?it/s]

In [42]:
# in case we are loading from disk, uncomment
# from llama_index.core import load_index_from_storage
# index2 = load_index_from_storage(storage_context)

#### Test created embedding/chunks


### Specify LLM-Chat interface
Now, we want to build the communication between an LLM and our database that resembles our typical RAG setup:
![Typical RAG pipeline](RAG_pipeline.png)



Using LlamaIndex, this is deceptively easy.

#### Specify details about retrieval from vector db

In [45]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

import logging
import sys

# Let's do some logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Configure retrieval from VectorDB 

In [46]:
# this specifies the details for retrieving the k closest elements to the user query
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5, # how many documents should we consider? Let's do 5
    verbose=True
)

#### Specify the used LLM
in this case, we use the OpenAI GPT4o-mini (very performant and cheap)

In [ ]:
# %pip install llama-index-llms-openai

In [52]:
from llama_index.llms.openai import OpenAI
# this is an OpenAI wrapper for llama_index
llm = OpenAI(model="gpt-4o-mini") 

#### Specify the prompt

Do a simple [RAG-prompt](https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/)

In [59]:
from llama_index.core import PromptTemplate
from llama_index.core import get_response_synthesizer

In [60]:
# Let's specify a prompt similar to what we have learned earlier
custom_query = """
    You are an information chatbot that informs users about the Interdisciplinary Transformation University Austria (ITU) in Linz, Austria. 
    
    Here is the context information:
    ---------------------
    {context_str}
    ---------------------
    Given the context information, this prompt, and no prior knowledge, answer the query. 
    The answer must be 100 words or less.
    
    Query: {query_str}
    Answer: """

In [61]:
# this specifies how we utilize the retrieved chunks/text in the response
# configure response synthesizer
rag_prompt = PromptTemplate(custom_query) # use LLama_index wrapper to create our query

# Build response synthesizer:
# i.e., object that combines user prompt, retrieved context, and our RAG prompt and sends it to the LLM (GPT-4o-mini)
response_synthesizer = get_response_synthesizer(
    llm=llm, text_qa_template=rag_prompt, verbose=True)

#### "Assemble" query engine 
Combine other config into the actual logic that will do the querying for us.

Again, we will stick to the basics here.


In [63]:
from llama_index.core.query_engine import RetrieverQueryEngine
#node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever, # configuration for retrieval of vector chunks
    response_synthesizer=response_synthesizer, # config for synthesizing LLM prompt/response
)

#### Let's prompt away!

In [68]:
oai_response = query_engine.query('Who is the founding president?')

In [88]:
oai_response.response

'The Founding President of the Interdisciplinary Transformation University Austria (ITU) is Prof. Dr. Stefanie Lindstaedt.'

In [125]:
# which files were used as context information? 
oai_response.metadata

{'542fde3f-28d1-4690-a8a9-dd7722665809': {'file_path': '/home/jovyan/work/preprocessed/en_public-notice_provisional-bylaws-of-itu-idsa.txt',
  'file_name': 'en_public-notice_provisional-bylaws-of-itu-idsa.txt',
  'file_type': 'text/plain',
  'file_size': 36144,
  'creation_date': '2024-08-26',
  'last_modified_date': '2024-08-26'},
 '2b55da0d-4516-441d-bc60-44fb958bd8c3': {'file_path': '/home/jovyan/work/preprocessed/en_digital-transformation-university_organization.txt',
  'file_name': 'en_digital-transformation-university_organization.txt',
  'file_type': 'text/plain',
  'file_size': 2374,
  'creation_date': '2024-08-26',
  'last_modified_date': '2024-08-26'},
 '81983a30-d6ca-4fce-a39b-6b0ae3c756c5': {'file_path': '/home/jovyan/work/preprocessed/en_public-notice_provisional-bylaws-of-itu-idsa.txt',
  'file_name': 'en_public-notice_provisional-bylaws-of-itu-idsa.txt',
  'file_type': 'text/plain',
  'file_size': 36144,
  'creation_date': '2024-08-26',
  'last_modified_date': '2024-08-2

In [126]:
# some more details
example_node_id = oai_response.source_nodes[1].node_id

print(f'Gathered information from {len(oai_response.source_nodes)} text chunks, for example:\n'
      f'Node ID: {example_node_id}\n'
      f'Document: {oai_response.metadata[example_node_id]["file_name"]}\n'
      f'Text:\n{oai_response.source_nodes[1].text}')

Gathered information from 3 text chunks, for example:
Node ID: 2b55da0d-4516-441d-bc60-44fb958bd8c3
Document: en_digital-transformation-university_organization.txt
Text:
:study:careerhome : about : organization
© Felix Büchele - IT:U:organization© Lunghammer – TU GrazDipl.-Ing.in Claudia von der Linden, MBA (IMD)Chairwoman of the Founding Convent© Antje Wolm – IT:UProf. Dr.in Stefanie LindstaedtFounding President© Felix Büchele – IT:UGabriele Költringer, EMBAManaging Directorinternational strategic advisory boardfounding conventfounding presidentfounding advisory boardmanaging directorFounding ConventThe Founding Convent is the strategic body of the university during the founding phase. Two of its members were nominated by the province of Upper Austria, three by the Federal Ministry of Education, Science and Research (BMBWF), two by the Federal Ministry for Climate Protection, Environment, Energy, Mobility, Innovation and Technology (BMK), one by the Austrian Science Fund (FWF) and one

#### Some other queries

In [69]:
oai_response2 = query_engine.query('Is there a summer school?')

In [86]:
oai_response2.response

"Yes, the Interdisciplinary Transformation University Austria (ITU) is hosting a Summer School in 2024, which has attracted over 200 applicants from 66 countries. The program emphasizes interdisciplinary collaboration and diverse academic backgrounds, with approximately 40 participants expected to be selected for this unique learning opportunity. The review committee is currently evaluating applications, and updates on the selection process will be provided as preparations continue. For more information, you can check the university's website."

In [82]:
oai_response3 = query_engine.query('Are they hiring?')

In [128]:
oai_response3.response

'Yes, the Interdisciplinary Transformation University Austria (ITU) is currently hiring. They have up to 12 postdoctoral positions available in the field of Computational X, as well as openings for a LMS Administrator – Full Stack Developer, Content Creator, Financial Controller, Project Controller, and Software Developer. Interested candidates can apply online and are encouraged to submit their applications, including a CV and cover letter. The application deadline for the postdoctoral positions is September 15th, 2024.'